# What's Cooking 4


In [75]:
# Trying to utilize some efficiency techniques I saw in the forums
from pandas import Series, DataFrame # I don't know how to use Pandas
import pandas as pd
import numpy as np 
import nltk #natural language toolkit
import re #regular expressions
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC #Let's see what linear SVC does
from sklearn.metrics import classification_report #not sure about this
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer # need to learn this too
from sklearn import grid_search
from sklearn.linear_model import LogisticRegression
import pickle

In [76]:
# Read in the data
train_data = pd.read_json("train.json")
print train_data.head() #great, we can use .head() with pandas! 

       cuisine     id                                        ingredients
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
2     filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...
3       indian  22213                [water, vegetable oil, wheat, salt]
4       indian  13162  [black pepper, shallots, cornflour, cayenne pe...


In [77]:
# what else can we do? 
print train_data['id'].head()

0    10259
1    25693
2    20130
3    22213
4    13162
Name: id, dtype: int64


In [78]:
# Not sure what the following line does exactly, need to figure it out
train_data['ingredients_clean_string'] = [' , '.join(z).strip() for z in train_data['ingredients']]

In [79]:
print train_data.head()

       cuisine     id                                        ingredients  \
0        greek  10259  [romaine lettuce, black olives, grape tomatoes...   
1  southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...   
2     filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...   
3       indian  22213                [water, vegetable oil, wheat, salt]   
4       indian  13162  [black pepper, shallots, cornflour, cayenne pe...   

                            ingredients_clean_string  
0  romaine lettuce , black olives , grape tomatoe...  
1  plain flour , ground pepper , salt , tomatoes ...  
2  eggs , pepper , salt , mayonaise , cooking oil...  
3               water , vegetable oil , wheat , salt  
4  black pepper , shallots , cornflour , cayenne ...  


In [80]:
print type(train_data['ingredients'][0])

<type 'list'>


In [81]:
# Looks like 'ingredients' contains a list of individual strings of ingredients
print train_data['ingredients'][0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']


In [82]:
### So then what does 'ingredients_clean_string' contain? 

In [83]:
print type(train_data['ingredients_clean_string'][0])
print train_data['ingredients_clean_string'][0]

<type 'unicode'>
romaine lettuce , black olives , grape tomatoes , garlic , pepper , purple onion , seasoning , garbanzo beans , feta cheese crumbles


In [84]:
# Looks like one continuous string. So that means that the above
# code added in a ' , ' between every ingredient and stuck it together. 
# What does .strip() do? 

In [85]:
# According to my research, .strip() with no arguments will remove leading
# and trailing whitespace (spaces), and if you specify characters, it 
# will remove those from the string if they're leading or trailing. 

In [86]:
test_string = 'this is a test string. com %'
print test_string
test_string_2 = test_string.strip('%')
print test_string_2
test_string_3 = test_string.strip(' tho')
print test_string_3

this is a test string. com %
this is a test string. com 
is is a test string. com %


In [87]:
# Great! I understand .strip(). so then .join() joins it all into a string? 
test_string_next = ' , test , '
print test_string_next.strip()

, test ,


In [88]:
test_list = ['a', 'b', 'd', 'u', 'test']
test_list_string = [' * '.join(j) for j in test_list]
print test_list_string

['a', 'b', 'd', 'u', 't * e * s * t']


In [89]:
test_list_string2 = [' * '.join(j).strip() for j in test_list]
print test_list_string2

['a', 'b', 'd', 'u', 't * e * s * t']


In [90]:
tester_list = ['audible', 'before ', 'dog test', 'understanding is key', 'test']
test_list_string3 = [' * '.join(j).strip() for j in tester_list]
print test_list_string3

['a * u * d * i * b * l * e', 'b * e * f * o * r * e *', 'd * o * g *   * t * e * s * t', 'u * n * d * e * r * s * t * a * n * d * i * n * g *   * i * s *   * k * e * y', 't * e * s * t']


In [91]:
# Looks like you put something iterable into .join() as an argument, 
# and that joins it all together for you. great! 
test_list_string4 = ' * '.join(tester_list)
print test_list_string4

audible * before  * dog test * understanding is key * test


In [92]:
train_data['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',\
                                                                                ' ', line)) 
                                           for line in lists]).strip() for lists in train_data['ingredients']]  

In [93]:
print train_data['ingredients_string'].head()
print train_data['ingredients_string'][0]
print type(train_data['ingredients_string'][0])

0    romaine lettuce black olives grape tomatoes ga...
1    plain flour ground pepper salt tomato ground b...
2    egg pepper salt mayonaise cooking oil green ch...
3                       water vegetable oil wheat salt
4    black pepper shallot cornflour cayenne pepper ...
Name: ingredients_string, dtype: object
romaine lettuce black olives grape tomatoes garlic pepper purple onion seasoning garbanzo beans feta cheese crumbles
<type 'unicode'>


In [94]:
corpus_train = train_data['ingredients_string']

In [95]:
# So let's look into what the above lines did. We get ' '.join, 
# that part adds together the items in the list with a space. 
# The interior part takes each set of ingredients, and then for each
# takes each individual ingredient string and does something with
# regular expressions and then lematizes that. So the questions is, 
# what does re.sub('[^A-Za-z]',' ', line) do? 

In [96]:
# Apparently ^ means that any character that doesn't fall into 
# the following category is matched. so Sub takes the string listed 
# as the third argument, tries to match anything listed in the first one
# (here as anything that's not a letter), and replaces that with a space

In [97]:
tst_strng = 'yes this is a test & woo h**y'
tst_strng2 = re.sub('[^A-Za-z]',' ', tst_strng)
print tst_strng2

yes this is a test   woo h  y


In [98]:
tst_strng3 = WordNetLemmatizer().lemmatize(tst_strng2)
print tst_strng3

yes this is a test   woo h  y


In [99]:
result = re.search('[^A-Za-z]', line) for line in lists for lists in train_data['ingredients']

SyntaxError: invalid syntax (<ipython-input-99-7575a2a6c82b>, line 1)

In [100]:
# What did I do wrong? No brackets around it
result = [(re.search('[^A-Za-z]', line)) for line in lists for lists in train_data['ingredients']]
print result[1:20]

[<_sre.SRE_Match object at 0x166ab1718>, <_sre.SRE_Match object at 0x166ab1648>, <_sre.SRE_Match object at 0x166ab1578>, <_sre.SRE_Match object at 0x166ab14a8>, <_sre.SRE_Match object at 0x166ab13d8>, <_sre.SRE_Match object at 0x166ab1308>, <_sre.SRE_Match object at 0x166ab1238>, <_sre.SRE_Match object at 0x166ab1168>, <_sre.SRE_Match object at 0x166ab1098>, <_sre.SRE_Match object at 0x166ab1780>, <_sre.SRE_Match object at 0x166ab17e8>, <_sre.SRE_Match object at 0x166ab1850>, <_sre.SRE_Match object at 0x166ab18b8>, <_sre.SRE_Match object at 0x166ab1920>, <_sre.SRE_Match object at 0x166ab1988>, <_sre.SRE_Match object at 0x166ab19f0>, <_sre.SRE_Match object at 0x166ab1a58>, <_sre.SRE_Match object at 0x166ab1ac0>, <_sre.SRE_Match object at 0x166ab1b28>]


In [101]:
print train_data['ingredients'][53]
print train_data['ingredients_string'][53]

[u'sugar', u'large eggs', u'all-purpose flour', u'baking soda', u'buttermilk', u'honey', u'corn oil', u'yellow corn meal', u'unsalted butter', u'salt']
sugar large eggs all purpose flour baking soda buttermilk honey corn oil yellow corn meal unsalted butter salt


In [102]:
# Looks like it just takes out non-numeric characters and makes them spaces. 

In [103]:
print train_data['ingredients'][68]
print train_data['ingredients_string'][68]

[u'orange juice concentrate', u'pumpkin pur\xe9e', u'marshmallow creme', u'toasted pecans', u'maple syrup', u'ground cinnamon', u'gingersnap', u'ground nutmeg', u'cream cheese']
orange juice concentrate pumpkin pur e marshmallow creme toasted pecans maple syrup ground cinnamon gingersnap ground nutmeg cream cheese


In [104]:
vectorizer_train = TfidfVectorizer(stop_words='english',
                             ngram_range = ( 1 , 1 ), analyzer="word", 
                             max_df = .57, binary=False, token_pattern=r'\w+', sublinear_tf=False)

In [105]:
tf_idf_train = vectorizer_train.fit_transform(corpus_train).todense()

### Test data

In [106]:
test_data = pd.read_json("test.json") 
test_data['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]',\
                                                                                  ' ', line)) 
                                             for line in lists]).strip() for lists in test_data['ingredients']]

In [107]:
corpus_test = test_data['ingredients_string']
tf_idf_test = vectorizer_train.transform(corpus_test)

predictors_train = tf_idf_train
targets_train = train_data['cuisine']
predictors_test = tf_idf_test

In [108]:
parameters = {'C':[1, 10]}
clf = LogisticRegression()
classifier = grid_search.GridSearchCV(clf, parameters)
classifier = classifier.fit(predictors_train, targets_train)

In [109]:
predictions = classifier.predict(predictors_test)
test_data['cuisine'] = predictions

test_data[['id' , 'cuisine' ]].to_csv("submission_2.csv")



In [110]:
# Learning about pickle
import pickle

save_classifier = open('saved_classifier_test_2.pickle', "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

# 0.78862